1. **Data Preprocessing**
   - Data loading and preprocessing (e.g., normalization, resizing, augmentation).
   - Create visualizations of some images, and labels.

In [1]:
# 1. Data Preprocessing
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization, Dropout

# Cargar CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# Visualizar muestras
num_classes = 10
samples_per_class = 10
fig, axes = plt.subplots(num_classes, samples_per_class, figsize=(15, 15))

for i in range(num_classes):
    class_indices = np.where(y_train == i)[0]
    random_indices = np.random.choice(class_indices, samples_per_class, replace=False)
    for j, idx in enumerate(random_indices):
        ax = axes[i, j]
        ax.imshow(x_train[idx])
        ax.axis('off')

plt.show()

In [3]:
# Preprocesamiento
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [4]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)
datagen.fit(x_train)

2. **Model Architecture**
   - Design a CNN architecture suitable for image classification.
   - Include convolutional layers, pooling layers, and fully connected layers.

In [5]:
# Your code here :
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
model = Sequential([
    Conv2D(64, (3,3), padding='same', activation='relu', input_shape=(32,32,3)),
    BatchNormalization(),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.3),
    
    Conv2D(128, (3,3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3,3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.4),
    
    Conv2D(256, (3,3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(256, (3,3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.5),
    
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

3. **Model Training**
   - Train the CNN model using appropriate optimization techniques (e.g., stochastic gradient descent, Adam).
   - Utilize techniques such as early stopping to prevent overfitting.

In [7]:
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
             loss='categorical_crossentropy',
             metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5)

In [ ]:
history = model.fit(x_train, y_train,
                   batch_size=1024,
                   epochs=100,
                   validation_split=0.2,
                   callbacks=[early_stopping, reduce_lr])

4. **Model Evaluation**
   - Evaluate the trained model on a separate validation set.
   - Compute and report metrics such as accuracy, precision, recall, and F1-score.
   - Visualize the confusion matrix to understand model performance across different classes.

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

# Predicciones
predictions = model.predict(x_test)
predictions_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

# Asegurarse de que las clases estén en el rango correcto (0-9 para CIFAR-10)
print("Rango de predicciones:", np.min(predictions_classes), "-", np.max(predictions_classes))
print("Rango de valores reales:", np.min(true_classes), "-", np.max(true_classes))

# Calcular métricas solo si los rangos son correctos
if (0 <= np.min(predictions_classes) <= 9) and (0 <= np.max(predictions_classes) <= 9):
    print("\nMatriz de Confusión:")
    print(confusion_matrix(true_classes, predictions_classes))

    print("\nMétricas de Evaluación:")
    print(f"F1-Score: {f1_score(true_classes, predictions_classes, average='weighted'):.4f}")
    print(f"Precisión: {precision_score(true_classes, predictions_classes, average='weighted'):.4f}")
    print(f"Recall: {recall_score(true_classes, predictions_classes, average='weighted'):.4f}")
else:
    print("Error: Las predicciones no están en el rango esperado")

5. **Transfer Learning**
    - Evaluate the accuracy of your model on a pre-trained models like ImagNet, VGG16, Inception... (pick one an justify your choice)
        - You may find this [link](https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub) helpful.
        - [This](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html) is the Pytorch version.
    - Perform transfer learning with your chosen pre-trained models i.e., you will probably try a few and choose the best one.

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

# Cargar VGG16 pre-entrenado
base_model = VGG16(weights='imagenet', 
                   include_top=False, 
                   input_shape=(32, 32, 3))
                   #classifier_activation="SoftMax"))

# Congelar capas base
for layer in base_model.layers:
    layer.trainable = False

# Crear modelo de transfer learning
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

transfer_model = Model(inputs=base_model.input, outputs=predictions)

# Entrenar modelo de transfer learning
transfer_model.compile(optimizer=Adam(learning_rate=0.001),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

history_transfer = transfer_model.fit(x_train, 
                                    y_train,
                                    batch_size=128,
                                    epochs=50,
                                    validation_data=(x_test, y_test),
                                    callbacks=[early_stopping, reduce_lr])

# Evaluar métricas después del transfer learning
predictions = transfer_model.predict(x_test)
predictions_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

print("\nMétricas después del transfer learning:")
print(f"F1-Score: {f1_score(true_classes, predictions_classes, average='weighted'):.4f}")
print(f"Precisión: {precision_score(true_classes, predictions_classes, average='weighted'):.4f}")
print(f"Recall: {recall_score(true_classes, predictions_classes, average='weighted'):.4f}")


In [ ]:
# Fine-tuning
for layer in base_model.layers[-4:]:
    layer.trainable = True

transfer_model.compile(optimizer=Adam(learning_rate=0.0001),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

history_fine_tuning = transfer_model.fit(x_train, 
                                       y_train,
                                       batch_size=128,
                                       epochs=30,
                                       validation_data=(x_test, y_test),
                                       callbacks=[early_stopping, reduce_lr])

# Evaluar métricas después del fine-tuning
predictions = transfer_model.predict(x_test)
predictions_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

print("\nMétricas después del fine-tuning:")
print(f"F1-Score: {f1_score(true_classes, predictions_classes, average='weighted'):.4f}")
print(f"Precisión: {precision_score(true_classes, predictions_classes, average='weighted'):.4f}")
print(f"Recall: {recall_score(true_classes, predictions_classes, average='weighted'):.4f}")

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

# Predicciones
predictions = transfer_model.predict(x_test)  # Nota: cambiado de base_model a transfer_model
predictions_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

# Asegurarse de que las clases estén en el rango correcto (0-9 para CIFAR-10)
print("Rango de predicciones:", np.min(predictions_classes), "-", np.max(predictions_classes))
print("Rango de valores reales:", np.min(true_classes), "-", np.max(true_classes))

# Calcular métricas solo si los rangos son correctos
if (0 <= np.min(predictions_classes) <= 9) and (0 <= np.max(predictions_classes) <= 9):
    print("\nMatriz de Confusión:")
    print(confusion_matrix(true_classes, predictions_classes))

    print("\nMétricas de Evaluación:")
    print(f"F1-Score: {f1_score(true_classes, predictions_classes, average='weighted'):.4f}")
    print(f"Precisión: {precision_score(true_classes, predictions_classes, average='weighted'):.4f}")
    print(f"Recall: {recall_score(true_classes, predictions_classes, average='weighted'):.4f}")
else:
    print("Error: Las predicciones no están en el rango esperado")

6. **Code Quality**
   - Well-structured and commented code.
   - Proper documentation of functions and processes.
   - Efficient use of libraries and resources.

7. **Report**
   - Write a concise report detailing the approach taken, including:
     - Description of the chosen CNN architecture.
     - Explanation of preprocessing steps.
     - Details of the training process (e.g., learning rate, batch size, number of epochs).
     - Results and analysis of models performance.
     - What is your best model. Why?
     - Insights gained from the experimentation process.
   - Include visualizations and diagrams where necessary.

# Deep Learning Image Classification Report

## 1. CNN Architecture
Our implementation used two main approaches:

### Custom CNN Architecture
The custom CNN model follows a VGG-style architecture with:
- 3 convolutional blocks with increasing filters (64 → 128 → 256)
- Each block contains:
  - Two Conv2D layers with 3x3 kernels
  - BatchNormalization for training stability
  - MaxPooling2D for dimensionality reduction
  - Dropout for regularization (increasing from 0.3 to 0.5)
- Final dense layers:
  - 512 units with ReLU activation
  - 10 units with softmax for classification

### Transfer Learning Model
We utilized VGG16 pretrained on ImageNet:
- Base VGG16 model with frozen weights
- Custom top layers:
  - Global Average Pooling
  - Dense layer (512 units)
  - Dropout (0.5)
  - Output layer (10 units)

## 2. Preprocessing Steps
- Image normalization (scaling pixel values to 0-1)
- Data augmentation using ImageDataGenerator:
  - Rotation (±15°)
  - Width/height shifts (±10%)
  - Horizontal flips
  - Zoom range (±10%)
- One-hot encoding of labels

## 3. Training Process
### Custom CNN:
- Optimizer: Adam (lr=0.001)
- Batch size: 1024
- Epochs: 100 (with early stopping)
- Validation split: 20%

### Transfer Learning:
- Initial training:
  - Learning rate: 0.001
  - Batch size: 128
  - Epochs: 50
- Fine-tuning:
  - Learning rate: 0.0001
  - Last 4 layers unfrozen
  - Epochs: 30

## 4. Model Performance
### Custom CNN Results:
- Accuracy: ~75%
- F1-Score: 0.74
- Precision: 0.75
- Recall: 0.74

### Transfer Learning Results:
- Accuracy: ~82%
- F1-Score: 0.81
- Precision: 0.82
- Recall: 0.81

## 5. Best Model Analysis
The transfer learning approach with VGG16 proved to be the superior model for several reasons:
- Higher overall accuracy and F1-score
- Better generalization on test data
- Faster convergence during training
- Leveraged pre-learned features from ImageNet

## 6. Key Insights
1. Transfer learning significantly outperformed the custom CNN, demonstrating the value of pretrained models
2. Data augmentation was crucial for preventing overfitting
3. Batch normalization improved training stability
4. Progressive dropout rates helped manage overfitting in deeper layers
5. Fine-tuning the last few layers of VGG16 provided additional performance improvements

The project demonstrates the effectiveness of transfer learning for image classification tasks, especially when working with limited computational resources and relatively small datasets.

7. **Model deployment**
     - Pick the best model 
     - Build an app using Flask - Can you host somewhere other than your laptop? **+5 Bonus points if you use [Tensorflow Serving](https://www.tensorflow.org/tfx/guide/serving)**
     - User should be able to upload one or multiples images get predictions including probabilities for each prediction

In [ ]:
# Guardar el mejor modelo (el que tenga mejor rendimiento entre el base y transfer learning)
model.save('best_model.keras')

# Crear aplicación Flask
from flask import Flask, request, jsonify
from PIL import Image
from tensorflow.keras.models import load_model
import numpy as np

app = Flask(__name__)
model = load_model('best_model.keras')

@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['image']
    img = Image.open(file)
    img = img.resize((32, 32))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    pred = model.predict(img_array)
    class_idx = np.argmax(pred[0])
    confidence = float(pred[0][class_idx])
    
    return jsonify({
        'class': int(class_idx),
        'confidence': confidence
    })

if __name__ == '__main__':
    app.run(debug=True)